In [1]:
cd ..

C:\Users\User\Documents\SURF 2021\RL RNN SURF\src


In [2]:
import sys
from multiprocessing.pool import Pool
from actionflow.data.data_process import DataProcess
from actionflow.rnn.lstm_beh import LSTMBeh
from actionflow.rnn.opt_beh import OptBEH
from actionflow.util.helper import cv_list, get_total_pionts
from actionflow.util.logger import LogFile, DLogger
from GL.data.data_reader import DataReader
from GL.util.paths import Paths
import tensorflow as tf

c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\user\documents\surf

In [25]:
cv_counts = 10
cv_lists = {}
data = DataReader.read_GL()
ids=list(data.keys())
cv_lists = cv_list(ids, cv_counts)

In [31]:
configs = []

for lr in [1e-2]:
    for cells in [5, 10, 20, 30]:
        for i in range(cv_counts):
            configs.append({'lr': lr, 'cells': cells, 'cv_index': i})

In [35]:
def run_GL_RNN(i):
    tf.reset_default_graph()
    ncells = configs[i]['cells']
    learning_rate = configs[i]['lr']
    cv_index = configs[i]['cv_index']

    output_path = Paths.local_path + 'GL/rnn-cv/' + str(ncells) + 'cells/' + 'fold' + str(cv_index) + '/'
    with LogFile(output_path, 'run.log'):
        indx_data = cv_lists_group[group][cv_index]
        gdata = data.loc[data.diag == group]
        train, test = DataProcess.train_test_between_subject(gdata, indx_data,
                                                             [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

        indx_data.to_csv(output_path + 'train_test.csv')
        train_merged = DataProcess.merge_data(train)
        DLogger.logger().debug("total points: " + str(get_total_pionts(train_merged)))
        del train

        worker = LSTMBeh(2, 0, n_cells=ncells)
        OptBEH.optimise(worker, output_path, train_merged, None,
                        learning_rate=learning_rate,
                        global_iters=3000,
                        load_model_path='../inits/rnn-init/' + str(ncells) + 'cells/model-final/'
                        )

In [37]:
Paths.local_path

'../nongit/results/local/'

In [36]:
run_GL_RNN(0)

2021-07-06 12:13:42,630 - DeepRL - DEBUG - version control: (None, None)


NameError: name 'cv_lists_group' is not defined